## Importação de Bibliotecas

In [1]:
import pymongo 
from pymongo import MongoClient
import pandas as pd
import numpy as np

## Conector Mongo Atlas

In [2]:
# conector do mongo atlas
uri = "mongodb+srv://giovana-soulcode.quvpom8.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority"
client = MongoClient(uri, tls=True, tlsCertificateKeyFile='/content/drive/MyDrive/X509-cert-7346878477947766211.pem')

In [3]:
# escolha do database e colecao
db = client['sample_supplies']
colecao = db['sales']

In [4]:
# verificar conexão / Qtd. documentos da coleção
colecao.count_documents({})

5000

In [5]:
for x in colecao.find():
  print(x)

Output hidden; open in https://colab.research.google.com to view.

## Extrair do Mongo

In [6]:
# extrair de uma colecao
cursor = colecao.find({})
dfmongo = pd.DataFrame(list(cursor))

In [7]:
dfmongo

,_id,saleDate,items,storeLocation,customer,couponUsed,purchaseMethod
0,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506,"[{'name': 'printer paper', 'tags': ['office', ...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online
1,5bd761dcae323e45a93ccfe9,2015-08-25 10:01:02.918,"[{'name': 'envelopes', 'tags': ['stationary', ...",Seattle,"{'gender': 'M', 'age': 50, 'email': 'keecade@h...",False,Phone
2,5bd761dcae323e45a93ccfea,2017-06-22 09:54:14.185,"[{'name': 'notepad', 'tags': ['office', 'writi...",Denver,"{'gender': 'M', 'age': 51, 'email': 'worbiduh@...",False,In store
3,5bd761dcae323e45a93ccfeb,2015-02-23 09:53:59.343,"[{'name': 'binder', 'tags': ['school', 'genera...",Seattle,"{'gender': 'F', 'age': 45, 'email': 'vatires@t...",False,In store
4,5bd761dcae323e45a93ccfec,2017-12-03 18:39:48.253,"[{'name': 'backpack', 'tags': ['school', 'trav...",London,"{'gender': 'M', 'age': 40, 'email': 'dotzu@ib....",False,In store
...,...,...,...,...,...,...,...
4995,5bd761deae323e45a93ce36b,2016-01-20 17:29:10.225,"[{'name': 'pens', 'tags': ['writing', 'office'...",Austin,"{'gender': 'F', 'age': 45, 'email': 'du@rum.py...",False,In store
4996,5bd761deae323e45a93ce36c,2014-11-29 10:09:41.573,"[{'name': 'pens', 'tags': ['writing', 'office'...",Denver,"{'gender': 'F', 'age': 47, 'email': 'dup@katje...",False,In store
4997,5bd761deae323e45a93ce36d,2017-12-25 07:02:51.283,"[{'name': 'binder', 'tags': ['school', 'genera...",Denver,"{'gender': 'M', 'age': 29, 'email': 'losuhse@e...",False,In store
4998,5bd761deae323e45a93ce36e,2013-12-07 11:06:25.510,"[{'name': 'notepad', 'tags': ['office', 'writi...",Denver,"{'gender': 'M', 'age': 59, 'email': 'hu@pehabu...",False,Online


In [8]:
dfmongo.dtypes

_id                       object
saleDate          datetime64[ns]
items                     object
storeLocation             object
customer                  object
couponUsed                  bool
purchaseMethod            object
dtype: object

## Tratamento do DF

### Transformar array em novas colunas

In [19]:
dfmongo.loc[0, 'customer']

{'gender': 'M', 'age': 42, 'email': 'cauho@witwuta.sv', 'satisfaction': 4}

In [20]:
dfcustomer = dfmongo['customer']
dfcustomer

0       {'gender': 'M', 'age': 42, 'email': 'cauho@wit...
1       {'gender': 'M', 'age': 50, 'email': 'keecade@h...
2       {'gender': 'M', 'age': 51, 'email': 'worbiduh@...
3       {'gender': 'F', 'age': 45, 'email': 'vatires@t...
4       {'gender': 'M', 'age': 40, 'email': 'dotzu@ib....
                              ...                        
4995    {'gender': 'F', 'age': 45, 'email': 'du@rum.py...
4996    {'gender': 'F', 'age': 47, 'email': 'dup@katje...
4997    {'gender': 'M', 'age': 29, 'email': 'losuhse@e...
4998    {'gender': 'M', 'age': 59, 'email': 'hu@pehabu...
4999    {'gender': 'M', 'age': 33, 'email': 'lihzodud@...
Name: customer, Length: 5000, dtype: object

In [22]:
customerdic = dfcustomer.loc[0]
customerdic.keys()

dict_keys(['gender', 'age', 'email', 'satisfaction'])

In [23]:
customerdic = dfcustomer.loc[0]
colnames = customerdic.keys()
newdf = pd.DataFrame(columns=colnames)

In [24]:
newdf

,gender,age,email,satisfaction


In [25]:
for i in range(5000):
  customer = dfcustomer.loc[i]
  newdf = newdf.append(customer, ignore_index=True)

newdf

,gender,age,email,satisfaction
0,M,42,cauho@witwuta.sv,4
1,M,50,keecade@hem.uy,5
2,M,51,worbiduh@vowbu.cg,5
3,F,45,vatires@ta.pe,3
4,M,40,dotzu@ib.sh,4
...,...,...,...,...
4995,F,45,du@rum.py,3
4996,F,47,dup@katjewtep.hr,3
4997,M,29,losuhse@eronehjod.np,4
4998,M,59,hu@pehabur.gd,4


### Drop de colunas

In [9]:
# drop colunas que não serão utilizadas da df
# dfmongo.drop(['_id'], axis=1, inplace=True)

In [10]:
# dfmongo.head(1)

,saleDate,items,storeLocation,customer,couponUsed,purchaseMethod
0,2015-03-23 21:06:49.506,"[{'name': 'printer paper', 'tags': ['office', ...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online


## Load 

In [30]:
dfmongo.to_csv('dfsales.csv')

In [36]:
newdf.to_csv('dfcustomer.csv')

## Merge

In [32]:
dfsales = pd.read_csv('/content/dfsales.csv')

In [37]:
dfcustomer = pd.read_csv('/content/dfcustomer.csv')

In [38]:
dft = dfsales.merge(dfcustomer, how='outer')
dft.head(5)

,Unnamed: 0,_id,saleDate,items,storeLocation,customer,couponUsed,purchaseMethod,gender,age,email,satisfaction
0,0,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506,"[{'name': 'printer paper', 'tags': ['office', ...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online,M,42,cauho@witwuta.sv,4
1,1,5bd761dcae323e45a93ccfe9,2015-08-25 10:01:02.918,"[{'name': 'envelopes', 'tags': ['stationary', ...",Seattle,"{'gender': 'M', 'age': 50, 'email': 'keecade@h...",False,Phone,M,50,keecade@hem.uy,5
2,2,5bd761dcae323e45a93ccfea,2017-06-22 09:54:14.185,"[{'name': 'notepad', 'tags': ['office', 'writi...",Denver,"{'gender': 'M', 'age': 51, 'email': 'worbiduh@...",False,In store,M,51,worbiduh@vowbu.cg,5
3,3,5bd761dcae323e45a93ccfeb,2015-02-23 09:53:59.343,"[{'name': 'binder', 'tags': ['school', 'genera...",Seattle,"{'gender': 'F', 'age': 45, 'email': 'vatires@t...",False,In store,F,45,vatires@ta.pe,3
4,4,5bd761dcae323e45a93ccfec,2017-12-03 18:39:48.253,"[{'name': 'backpack', 'tags': ['school', 'trav...",London,"{'gender': 'M', 'age': 40, 'email': 'dotzu@ib....",False,In store,M,40,dotzu@ib.sh,4
